In [ ]:
#가상환경 cuda로 실행할 것

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

C:\Users\user\anaconda3\envs\cuda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\envs\cuda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\user\anaconda3\envs\cuda\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import torch
torch.__version__

'1.8.1+cu111'

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
df_newsgroup = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/20news_dataset_clear/20newsgroup_preprocessed.csv', sep=';', usecols=['target', 'text_cleaned'])
df_newsgroup.rename(columns={'text_cleaned' : 'text'}, inplace=True)

In [5]:
le = LabelEncoder()
le.fit(df_newsgroup['target'].unique())

LabelEncoder()

In [6]:
df_newsgroup['target'] = le.transform(df_newsgroup['target'])

In [7]:
X = df_newsgroup['text'].astype(str)
# y = tf.keras.utils.to_categorical(df_newsgroup['target'], num_classes=df_newsgroup['target'].nunique())
y=list(df_newsgroup['target'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df_newsgroup['target'])

In [8]:
from torch.utils.data import Dataset
import numpy as np

class TextCuboidDataset(Dataset):
    def __init__(self, data_dir, labels):
        self.data_dir = data_dir
        self.labels = labels
        self.data = []
        
        for i in range(14062):
            data_path = data_dir+'/cuboid%d.npy'%i
            self.data.append(np.load(data_path))
            
#         print (np.asarray(self.data).shape)
#         print (np.asarray(self.label).shape)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [9]:
from torch.utils.data import Dataset
import numpy as np

class ValidDataset(Dataset):
    def __init__(self, data_dir, labels):
        self.data_dir = data_dir
        self.labels = labels
        self.data = []
        
        for i in range(1000):
            data_path = data_dir+'/cuboid%d.npy'%(i+14062)
            self.data.append(np.load(data_path))
            
#         print (np.asarray(self.data).shape)
#         print (np.asarray(self.label).shape)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [10]:
from torch.utils.data import Dataset
import numpy as np

class TestDataset(Dataset):
    def __init__(self, data_dir, labels):
        self.data_dir = data_dir
        self.labels = labels
        self.data = []
        
        for i in range(3766):
            data_path = data_dir+'/cuboid_test%d.npy'%i
            self.data.append(np.load(data_path))
            
#         print (np.asarray(self.data).shape)
#         print (np.asarray(self.label).shape)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
#TextCuboid(Hub+NG) -> data_dir = 'D:/textcuboid_20ng_dual'
#TextCuboid(NG+AG) -> data_dir = 'D:/textcuboid_20ng_dual(ng+ag)'
#TextCuboid(Hub+AG) -> data_dir = 'D:/textcuboid_20ng_dual(hub+ag)'
data_dir = 'D:/textcuboid_20ng_dual'
dataset = TextCuboidDataset(data_dir, y_train[:14062])
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
valid = ValidDataset(data_dir, y_train[14062:])
validloader = DataLoader(valid, batch_size=128, shuffle=True)

In [11]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        filter_cnt = 1024
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=filter_cnt, kernel_size=(1, 1024))
        self.relu = nn.ReLU()
        self.pooling = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(filter_cnt, 512)
        self.fc2 = nn.Linear(512, 128)
        self.output_layer = nn.Linear(128, 20)

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(-1,x.shape[1],x.shape[2]*x.shape[3])
        x = self.pooling(x).view(-1,x.shape[1])
        
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

#         out=self.output_layer(x)
        return self.output_layer(x)

model=CNN().cuda()

In [12]:
import torchsummary
torchsummary.summary(model,input_size=(2,1000,1024))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 1024, 1000, 1]       2,098,176
              ReLU-2        [-1, 1024, 1000, 1]               0
 AdaptiveMaxPool1d-3              [-1, 1024, 1]               0
            Linear-4                  [-1, 512]         524,800
              ReLU-5                  [-1, 512]               0
            Linear-6                  [-1, 128]          65,664
              ReLU-7                  [-1, 128]               0
            Linear-8                   [-1, 20]           2,580
Total params: 2,691,220
Trainable params: 2,691,220
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 7.81
Forward/backward pass size (MB): 15.64
Params size (MB): 10.27
Estimated Total Size (MB): 33.72
----------------------------------------------------------------


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNN(
  (conv1): Conv2d(2, 1024, kernel_size=(1, 1024), stride=(1, 1))
  (relu): ReLU()
  (pooling): AdaptiveMaxPool1d(output_size=1)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (output_layer): Linear(in_features=128, out_features=20, bias=True)
)

In [15]:
total_batch = len(dataloader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 110


In [16]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters())

num_epochs = 10
best_val_acc = 0.0

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (data,label) in enumerate(dataloader):
        data = data.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    # 검증
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for i, (data,label) in enumerate(validloader):
            data = data.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.long)

            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            val_total += label.size(0)
            val_correct += (predicted == label).sum().item()
    
    val_acc=(100 * val_correct / val_total)
    print(f'Epoch: {epoch+1}/{num_epochs} Loss: {running_loss/len(dataloader)} Accuracy: {100 * correct / total} Val Accuracy: {val_acc}')
#     print(f'Epoch: {epoch+1}/{num_epochs} Loss: {running_loss/len(dataloader)} Accuracy: {100 * correct / total}')

    # 검증 정확도가 가장 높은 모델 저장
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'D:/save model/dual_best_model.pt')

Epoch: 1/10 Loss: 1.3609846591949464 Accuracy: 57.3460389702745 Val Accuracy: 73.3
Epoch: 2/10 Loss: 0.6717881273139606 Accuracy: 78.97169677144076 Val Accuracy: 76.6
Epoch: 3/10 Loss: 0.43012711297382006 Accuracy: 86.38884938131133 Val Accuracy: 75.6
Epoch: 4/10 Loss: 0.24855137169361113 Accuracy: 92.50462238657374 Val Accuracy: 78.0
Epoch: 5/10 Loss: 0.15210925824940205 Accuracy: 95.27094296686104 Val Accuracy: 79.0
Epoch: 6/10 Loss: 0.10806188701905987 Accuracy: 96.89944531361115 Val Accuracy: 78.7
Epoch: 7/10 Loss: 0.09716141862286763 Accuracy: 97.19101123595506 Val Accuracy: 78.2
Epoch: 8/10 Loss: 0.07367420630021529 Accuracy: 97.9021476319158 Val Accuracy: 77.8
Epoch: 9/10 Loss: 0.08682059808210893 Accuracy: 97.44702033850092 Val Accuracy: 77.4
Epoch: 10/10 Loss: 0.0742378538643772 Accuracy: 97.8025885364813 Val Accuracy: 79.1


In [17]:
torch.save(model.state_dict(), 'D:/save model/dual_cnn_model.pt')

------------

In [14]:
#마지막 모델
model.load_state_dict(torch.load('D:/save model/dual_cnn_model.pt'))

<All keys matched successfully>

In [14]:
#베스트 모델
model.load_state_dict(torch.load('D:/save model/dual_best_model.pt'))

<All keys matched successfully>

In [15]:
#TextCuboid(Hub+NG) -> test_dir = 'D:/textcuboid_20ng_dual_test'
#TextCuboid(NG+AG) -> test_dir = 'D:/textcuboid_20ng_dual(ng+ag)_test'
#TextCuboid(Hub+AG) -> test_dir = 'D:/textcuboid_20ng_dual(hub+ag)_test'
test_dir = 'D:/textcuboid_20ng_dual_test'
testset = TestDataset(test_dir, y_test)
testloader = DataLoader(testset, batch_size=128)

In [15]:
test_dir = 'D:/textcuboid_20ng_dual_test(ng+ag)'
testset = TestDataset(test_dir, y_test)
testloader = DataLoader(testset, batch_size=128)

In [15]:
test_dir = 'D:/textcuboid_20ng_dual_test(hub+ag)'
testset = TestDataset(test_dir, y_test)
testloader = DataLoader(testset, batch_size=128)

In [16]:
labels = []
predicteds = []

model.eval ()

with torch.no_grad ():
    for i, (data, label) in enumerate (testloader):
        
        data = data.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.long)
        
        output = model (data)
        
        _, predicted = torch.max (output, 1)
        accuracy = (label==predicted.squeeze()).float().mean()
        
        labels.extend (label.cpu())
        predicteds.extend (predicted.squeeze().cpu())
        
        print ('accuracy: {:.4f}'.format (accuracy.item()))

accuracy: 0.8438
accuracy: 0.7734
accuracy: 0.7734
accuracy: 0.8125
accuracy: 0.7812
accuracy: 0.7500
accuracy: 0.8203
accuracy: 0.7891
accuracy: 0.7188
accuracy: 0.7656
accuracy: 0.8516
accuracy: 0.8125
accuracy: 0.7500
accuracy: 0.8359
accuracy: 0.8281
accuracy: 0.7812
accuracy: 0.8125
accuracy: 0.8203
accuracy: 0.7891
accuracy: 0.7812
accuracy: 0.7656
accuracy: 0.7656
accuracy: 0.7891
accuracy: 0.7812
accuracy: 0.7578
accuracy: 0.7734
accuracy: 0.8359
accuracy: 0.8516
accuracy: 0.7891
accuracy: 0.8148


In [17]:
from sklearn.metrics import f1_score
from sklearn import metrics

print(metrics.classification_report(labels, predicteds, digits=3))

              precision    recall  f1-score   support

           0      0.779     0.750     0.764       160
           1      0.692     0.749     0.719       195
           2      0.645     0.858     0.736       197
           3      0.747     0.571     0.647       196
           4      0.715     0.771     0.742       192
           5      0.875     0.714     0.787       196
           6      0.667     0.732     0.698       194
           7      0.748     0.884     0.810       198
           8      0.925     0.869     0.896       199
           9      0.866     0.910     0.887       199
          10      0.968     0.905     0.935       200
          11      0.927     0.838     0.881       198
          12      0.846     0.561     0.675       196
          13      0.910     0.864     0.886       198
          14      0.861     0.848     0.854       197
          15      0.746     0.850     0.794       200
          16      0.812     0.808     0.810       182
          17      0.945    

In [18]:
del model
torch.cuda.empty_cache()